*Report Basics of Mobile Robotics*  
## Students: 
- Levy Alexandre 
- Tourki Amine 
- Tourki Emna 
- Zghari Aya






Link to our [Github](https://github.com/AmineTourki/Mobile-Robotics-)

## Introduction
This project aims to make a Thymio navigate in a course where it will have to use Computer Vision, Global and Local Navigation with a filter to reduce positioning errors.

The arena that we present you is modular, the obstacles can change place and the start and goal can be completely different. In this arena we also force our robot to pass through a checkpoint, which will spice up the route it has to make.The Thymio first goes to the checkpoint in green and then to the final point in red.

## Method Overview 
![alt text](report_images/Arena_axis.png "Title") 
**Main ideas:**
- The "global obstacles" are the black 2D rectangles on the ground detected by the vision.
- The "Local obstacles" are physical 3D obstacles detected by the proximity sensors and will be intoduced while the robot moves along the global path. We chose a cylindrical white obstacle for the presentation.  
- Our robot cannot move through an arc of circle.It can only move straight forward or rotate in place. However , the navigation can still be smooth if we use an 8-Direction A_Star path planning and take small steps forward.That's why in this project we used an 8-Direction A_Star path planning and we set a small fixed time step to go forward.  

GLOBAL_FORWARD = 0.1  # step time (in seconds) spent going forward in global navigation.  
LOCAL_FORWARD = 0.1 # step time (in seconds) spent going forward in local navigation.

- We chose a speed of MOTORSPEED = 50 for both the rotation and and the forward movement.
- We used a Kalman filter.We use simple odometry to estimate the robot position and the camera to correct the robot position.Thus, the camera will be used continuously.
- The global axes are defined as in the image above and the absolute angle of the robot is then defined positive if clockwise. 

## Vision

### Map Setup

<img src="Images/Setup.png" width="600"/>

### Implementation details

The vision Class is strongly dependent on the map data, which can be updated continuously from the video. The construction of a global map consists of the following steps:

1. The raw images are sampled at a given frequency from the webcam.
2. A rebounding of the arena is done.
3. Multiple color masks are applied to the transformed image to distinguish the objectives,the thymio, the arena and obstacles.
4. Multiple color masks are applied to the transformed image to distinguish the target position, thymio (front and rear) and obstacles.

Note:
The rebounding and the localisation of the objectives and the obstacle are done on the first image. The next images are used for the robot position given to the Kalman filter.   
The open library CV2 was used for the vision module.


- Input
    
    - Raw image captured from camera
    - Distance in real between the two circles on the robot to have a real world scale
    - Color HSV values that helps define the map (this is sensitive to light and needs to be calibrated after setting environment)
    
- Output
    - Global map that marks all occupancy with 1 and vacanncy with 0
    - Starting pose, checkpoint pose and finish pose of the Thymio

- Limitations 
    - The color masks are rather sensitive to lighting conditions and hyperparameters had to be calibrated
    
- Keyparameters
    - Color mask Table
        | Color | Used for | Lower Bounds | Upper Bounds |
        | --- | --- | --- | --- |
        | Blue | Robot Position | [87,97,50] | [164,255,160] |
        | Red | Final Destination | [150,70,50] | [190,255,255] |
        | Green | Checkpoint | [40,40,40] | [100,255,255] |
        | Black | Robot Position | [0,0,0] | [180,255,50] |
        | White+Green+Blue+Red | Reboarder Map |  [0,0,40] | [255,255,255] |



### Class Vision

Here is the class that can be found in vision_agent.py and its initialization:

    class Vision_Agent:
    def __init__(self):
        self.converter = 0
        self.cam = cv2.VideoCapture(1)
        ret, frame = self.cam.read()  # initialize
        self.image = frame
        self.resize()                 #resize the image
        self.hsv = cv2.cvtColor(self.image, cv2.COLOR_BGR2HSV) # convert to HSV
        self.get_first_image_cam(self.image)                   # delimit of the image
        self.hsv = cv2.cvtColor(self.image, cv2.COLOR_BGR2HSV)
        
        
        self.angle = 0              # angle of the robot
        self.center_robot = [0,0]   # center of the robot
        self.parcours = np.zeros((50,50), np.uint8) # occupancy grid
        self.parcours_2_pix=[len(self.image[0])/len(self.parcours[0]),len(self.image)/len(self.parcours)]
        self.objective_red = None
        self.objective_green = None
        self.r_in_pix = 0
        self.r_in_real = 6

The Object Vision is going to initialise all these values and we are going to call update methods and at any time, there are also methods for visualisation and others returning  coefficient values like the conversion from pixel to a real value.


Here is a photo example to illustrate the vision code : 

<img src="Images/Image13.jpeg" width="220"/>

In order to output a global map, each different object has to be detected. In order to detect the robot and its direction we put two blue rounds on it with different sizes, the small one is pointing in its direction. The black objects are the obstacles. The red represents the destination point. The green represents a checkpoint. 

#### Methods in the class

##### resize()

Resize the image returns by the camera so that the computation time is smaller. 

##### read_image()

Read a new image from the camera.

##### mask_thresh (image, thresh_low, thresh_high)

Make a mask of the image in the range specified by the thresholds.

#### redefine_mask

Apply Morphological Operations so that erases noise in the mask.


<img src="Images/Denoise.png" width="220"/>
<img src="Images/Boom.png" width="220"/>

##### get_robot()

Get the position of the robot in the image stored.

Here is a photo of the mask after mask_thresh with black and redefine mask for denoising

![alt text](Images/Mask_circles.png "Title")

Then we detect the circles with cv2 HoughCircle and Compute their centers

![alt text](Images/Image_Robot.png "Title")

##### get_obstacles()

Get the position of the bounding box of the obstacles and returns an occupancy grid for the A* algorithm



Here are photos of the masks after mask_thresh with black and redefine mask for denoising  
![alt text](Images/Mask_obstacles.png "Title")

Then we create a bounding box around each obstacle  
![alt text](Images/Image_obstacles.png "Title")

This parcours is returned as occupancy grid and the obstacles are also dilated so it is easier to dodge them   
<img src="Images/parcours_a_star.png" width="220"/>

##### get_objectives()

Get the center of the objective.

Here is a photo of the mask after mask_thresh with green and red and redefine mask for denoising  
![alt text](Images/Mask_objective_green.png "Title")
![alt text](Images/Mask_objective_red.png "Title")

Then we create a bounding box around each objective and get their center  
![alt text](Images/Image_Objectives.png "Title")


##### print_path()

Print the path generated by the A* algorithm on the image.  

<img src="Images/Image50.jpeg" width="220"/>

##### get_first_image_cam()

Crop the image so that it fits the arena and store the coordinates of the arena.
 
It first creates a mask of the arena with mask_thresh and morphological Operations.  
<img src="Images/Image_reboardered.png" width="220"/>

Then we detect the angles of this mask and recompute a reboardered image from the angles

##### get_image_cam()

Use the coordinates of the arena to crop the Image .

##### get_pix_2_real()

Returns a factor to convert from pixel value to real value in cm.

##### get_grid_2_real ()

Returns a factor to convert from grid value to real value in cm.

##### update

Do  get_robot(), get_obstacles(), get_objectives(). All the necessary task to detect all the object on the map.


![alt text](Images/Image.png "Title")

## Navigation

To facilitate the navigation, we created a class that represents the robot and control its motors. The class can be found in utils.py

In [ ]:

class MyThymio(object):

    def __init__(self, th, x, y, theta):
        """
        Initialisation of the object
        :param th: instance linking to the Thymio robot connected via USB
               x,y: x and y positions of the center of the Thymio
               theta: the absolute angle orientation of the Thymio
        :return: creates the object of class MyThymio
        """

        self.th = th
        self.x = x
        self.y = y
        self.angle = normalise_angle(theta)


Here are the methods that we can call on an instance: 
- def get_pos(self)
- def set_pos(self, x, y)
- def motor_stop(self)
- def motor_forward(self, time_forward)
- def motor_rotate(self, dtheta)
- def get_angle(self)
- def set_angle(self, theta)
- def get_previous_pos(self)

*Notes:*  

- The method **motor_rotate(self, dtheta)** sets the speed of the motors to ±MOTORSPEED to make the robot turn on himself dtheta rad then updates self.angle. It takes into consideration the sign of dtheta. If dtheta positive/ negative, the robot turns right/left.
- The method **motor_forward(self, time_forward)** sets the speed of both motors to MOTORSPEED  for time_forward seconds.However this method don't update the positions x,y of the instance self. x,y of the instance self are only updated after estimating the true positions with the Kalman filter.They are updated each time the function **callKalman** is called.The definition of this function **callKalman** is in the file navigation.py
- The method **get_previous_pos(self)** is used in the function **callKalman**. Since x and y are only updated just after calling kalman, they are the previous positions of our robot when calling it.

### A-Star Path Planning

We use the same A* algorithm provied in the TP. It is based on the pseudocode provided in [Wikipedia](https://en.wikipedia.org/wiki/A*_search_algorithm)  
The vision module is responsible for calculating the occupancy grid, the size of the grid is modular and can be adjusted in the Vision module. For our project, we fixed a size of 50  by 50 for performance reasons. A 50 by 50 grid takes around 4 seconds to be resolved while a grid of 60 by 60 takes 12 seconds (can vary depending on the power of the computer) that is why we opted for that size.  
For a better looking and more smooth navigation, we also opted for 8-connectivity movement (8N)  
**Note:** The implementation of the code can be found in the file A_star_algorithm.py

for the given arena:    
![alt text](report_images/A_star_0.jpeg "Title")    
The vision would generate the following mask:    
![alt text](report_images/A_star_1.png "Title")    
and generate the following grid    
![alt text](report_images/A_star_2.png "Title")  
For the first checkpoint (in green) we generate the following path  
![alt text](report_images/visited_green.png "Title")  
And for the Goal (in red) we obtain the following path:  
![alt text](report_images/visited_red.png "Title")  

The following function was created to directly call the A star algorithm, it only takes as input the Start, the Goal and the occupency grid 

```def run_A_Star(occupancy_grid, start, goal):```

### Global navigation

The main loop (see implementation at the end of the report) follows this pseudo code:  

Repeat  
- construct path using function **run_A_Star**
- follow that path using function **navigation.follow_path**. Two possibilities to exit **the function follow path**:
    - First one: The robot follow the path until the end. The goal is reached. Exit **navigation.follow_path** with return True.
    - Second one: When following the path, a local obstacle is detected with the proximity sensors. The robot enters the local navigation , moves to avoid the obstacle locally then exit **navigation.follow_path** with return False to construct a new path with the A_Star.
- If first goal objective_green reached (checkpoint goal), change goal to objective_red (final destination)
- If red goal is reached, break the loop
        
        
        



Here are the function **navigation.follow_path** and the functions related to it. The definition of these function are imported from  the file navigation.py

In [ ]:

def closest_node_index(current_pos, path, old_index):
    """
    return the closest node of the path to the current position of the robot
    and if the robot should change his orientation

    :param current_pos: current position of the robot
    :param path: array containing the optimal path to follow
    :param old_index: index taken in the last call of this function
    :return:    closestnode_index (int)
                change_angle (bool)

    """
    # check the nodes from old index and ahead and get the index of the closest node to our current position
    closestnode_index = old_index + np.linalg.norm(current_pos - path[old_index:], axis=1).argmin()
    # only change the orientation if the robot is closest to one of his next nodes compared to the initial node
    # don't change angle if already close to goal (goal index ==len(path) - 1)
    change_angle = (closestnode_index != old_index) and (closestnode_index != len(path) - 1)
    return change_angle, closestnode_index


def change_orientation(myRobot, next_pos):
    """
    adjust the orientation of the robot by rotating him towards his next_pos
    :param myRobot: our robot, instance of Class MyThymio
    :param next_pos: coordinate of the next position
    """
    target_vector = next_pos - myRobot.get_pos()
    dtheta = math.atan2(target_vector[1], target_vector[0]) - myRobot.get_angle()
    # rotate the robot with angle dtheta
    myRobot.motor_rotate(dtheta)


def follow_path(myRobot, Vision, path):
    """
    Follow the path until the goal is reached or until the robot enters local navigation
    :param myRobot: our robot, instance of Class MyThymio
    :param Vision: our vision agent, instance of Class Vision_Agent
    :param path: array containing the optimal path to follow
    :return:    True if the goal is reached
                False if the robot entered local navigation (goal not reached)
    """
    closest_index = 0
    first_step = 1
    while math.dist(myRobot.get_pos(), path[-1]) > EPS_GOAL:  # path[-1] is the goal
        change_angle, closest_index = closest_node_index(myRobot.get_pos(), path, closest_index)
        # only change the orientation if the robot is closest to his next node compared to the initial node
        if change_angle or first_step:
            first_step = 0
            next_pos = path[closest_index + 1]
            # rotate to face the new target node next_pos
            change_orientation(myRobot, next_pos)
        # go forward GLOBAL_FORWARD(0.1) seconds
        myRobot.motor_forward(GLOBAL_FORWARD)
        # estimate new position and update myRobot pose
        callKalman(myRobot, Vision, GLOBAL_FORWARD)
        # enter local navigation if obstacle detected
        entered_local_navigation = avoid_obstacle(myRobot, Vision)
        if entered_local_navigation:
            # if the robot entered local navigation , return to the main implementation loop to construct a new path
            return False

    print("Goal reached")
    myRobot.motor_stop()
    return True

*Notes:*  

- In the function **closest_node_index(current_pos, path, old_index)**

### Local navigation using proximity sensors

## Filter

### Kalman Filter    
Our Kalman filter consists of a prediction step and a correction step. We use simple odometry to estimate the robot position and the camera to correct the robot position. The filter was also designed to work on linear movements only.  
The global navigation consists of translation steps and rotations steps done seperately, but we will assume that the rotations do not infer any error in the odometry. This choice was made because the camera is not sensitive enough to detect small rotations and would return false values. We also found that using only the translation is sufficient and the filter performs still well. We also use a constant speed of 50 for the Thymio.
The filter takes the position of the robot as well as its orientation as an input vector.  
All the calculus was made in centimeters for the robot positions and in radians for the angles  
**Note:** the implementation of the Kalman filter can be found in the file Kalman.py

$$X = \begin{bmatrix} x\\
y\\
\alpha
\end{bmatrix}$$

We takes an arbitrary covariance matrix with high values as we are uncertain of where the robot starts
$$P = \begin{bmatrix} 100 && 0 && 0\\
0 && 100 && 0\\
0 && 0 && \frac{\pi}{4}
\end{bmatrix}$$  





The odometry uses this dynamic equation:  

$$X_{priori}=\begin{bmatrix} x_{priori}\\
y_{priori}\\
\alpha_{priori}
\end{bmatrix} = \begin{bmatrix} x_{previous}\\
y_{previous}\\
\alpha_{previous}
\end{bmatrix} + 
\begin{bmatrix} speed*\Delta t*sin(\alpha_{previous})\\
speed*\Delta t*cos(\alpha_{previous})\\
0
\end{bmatrix}
$$  

For the computation of the new covariance matrix, we take the jacobian of the dynamic matrix.

$$A = \begin{bmatrix} 1 && 0 && -speed*\Delta t*sin(\alpha_{previous})\\
0 && 1 && -speed*\Delta t*sin(\alpha_{previous})\\
0 && 0 && 0
\end{bmatrix}$$

and we finally add the uncertainty matrix to our covariance matrix. we obtain:

\begin{equation}
P_{priori}=APA^{T} + Q
\end{equation}


For the matrix Q we use the variance of the robot speed to calculate a position variance using sampling, we obtain a value of  $\sigma_x$ = $\sigma_y$ = 0.123 cm. The error on the angle is set arbitrarily to $3^{\circ}$ = $\frac{\pi}{60}$ 

$$Q = \begin{bmatrix} 0.123 && 0 && 0\\
0 && 0.123 && 0\\
0 && 0 && \frac{\pi}{60}
\end{bmatrix}$$

Once the estimation step is complete, we use the update step to correct the robot position using the robot position returned by the camera.  
In this step we simply assume the noise R on the camera readings to be equal to $\sigma$ = 2*pixel size (cm). The value of the pixel size in centimeters is given by the the vision agent. We obtain diffent values of pixel size for the x and y values depending on the size of the Arena we are using. We also approximate the angle error on the camera to $4.5^{\circ}$ = $\frac{\pi}{40}$ 

$$R = \begin{bmatrix} \sigma_x && 0 && 0\\
0 && \sigma_y && 0\\
0 && 0 && \frac{\pi}{40}
\end{bmatrix}$$

The measurement matrix H is a unit diagonal matrix as we simply take the values returned by the vision module without modifications. 

$$H = \begin{bmatrix} 1 && 0 && 0\\
0 && 1 && 0\\
0 && 0 && 1
\end{bmatrix}$$

We then simply calculate the innovation $i$ and the prediction covariance S in order to calculate the Kalman gain K. The equations are as follow

\begin{equation}
i = X_{mesure} - HX_{priori}  
\end{equation}
\begin{equation}
S=HP_{priori}H^{T} + R
\end{equation}

The Kalman K gain tells how much the predictions should be corrected based on the measurements

\begin{equation}
K=P_{priori}H^{T}S^{-1}
\end{equation}

We can then calculate a posterior position estimate and a posterior covariance matrix estimate

\begin{equation}
X_{post} = X_{mesure} + ki  
\end{equation}
\begin{equation}
P_{post}=P_{priori} - KHP_{priori}
\end{equation}

Given that the image quality of the camera isn't very high, we can get errors on the measured values at random times. To make up for that we simply discard the measured values by the camera and only use the estimated values in the Kalman filter. A boolean variable (robot_detected) is simply set to True by the vision module in order for the Kalman filter to use the prediction only 

The final Kalman filter can be called by the function: 

```def kalman_estimate(X_previous,P_previous,delta_t,speed)```

it returns the corrected position and angle of the robot

\begin{equation}
X_{final}= kalman\_estimate(X\_previous,P\_previous,delta\_t,speed)
\end{equation}

## Imports 

In [ ]:
import numpy as np
import math as m
import cv2
import os
import sys
import matplotlib.pyplot as plt
import time
sys.path.insert(0, os.path.join(os.getcwd(), 'src'))
from vision_agent import Vision_Agent
import local_navigation
from A_star_algorithm import run_A_Star
import navigation
import utils
import  Kalman

%matplotlib inline

## Connexion to Thymio 

In [ ]:
from Thymio import Thymio
th = Thymio.serial(port="COM7", refreshing_rate=0.1)

## Main implementation to run the project


In [ ]:
# Get map information and Initialise Vision's instance
Vision=Vision_Agent()
Vision.read_save_image("image_start.png")
Vision.update() # get robot , obstacles and goals positions
#Initialize the robot's instance
myRobot=utils.MyThymio(th, Vision.center_robot[0],Vision.center_robot[1],utils.deg_to_rad(Vision.angle))
#Initialize Kalman
Kalman.kalman_init(Vision)
#Initialize Variables
checkPointReached=0
finalGoalReached=0
goal=Vision.objective_green

# Main loop
while True
    # construct/reconstruct the path to the goal
    Vision.path = run_A_Star(Vision.parcours, myRobot.get_pos(), goal)
    Vision.print_path() 
    
    goalReached=navigation.follow_path(myRobot,Vision,Vision.path)
    
    if goalReached :
        if checkPointReached == 0 :
            checkPointReached=1
            goal=Vision.objective_red
        else :
            print("Final destination reached")
            myRobot.motor_stop()
            break
            
Vision.cam.release()
